In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import sys

sys.path.append("../analysisTools/")
import plotTools as ptools
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#coffea output file
f_bkg = "../coffea/skimmed/HT_skimmed_WJets_2022.coffea"

#Nested Dictionary:
bkg_histo = util.load(f_bkg) [0]

In [5]:
cuts = list(bkg_histo['cutflow']['bkg_2022_WJets_WJetsToLNu6'])
tot_bkg_cts = {} 
tot_bkg_effs = {} 
total_bkg_counts = {}


cfd = bkg_histo['cutflow_cts']['bkg_2022_WJets_WJetsToLNu6']['all']
print("cutflow_cts:",cfd)
effd = bkg_histo['cutflow']
print(effd)
#effd = bkg_histo['cutflow']['bkg_2022_WJets_WJetsToLNu5']['all']
#print("cutflow:",effd)
nEvents=bkg_histo['cutflow_nevts']['bkg_2022_WJets_WJetsToLNu6']['all']
print("cutflow_nevts:",nEvents)
cutNames = {cname:ptools.getCut(bkg_histo['cutDesc'][cname]) for cname in bkg_histo['cutDesc'].keys()}
print(cutNames)

    

    

cutflow_cts: 15198.748808579854
{'bkg_2022_WJets_WJetsToLNu6': defaultdict(<class 'float'>, {'all': 0.05304403474365913, 'hasVtx': 0.0014652779262902783, 'cut1': 0.0014652779262902783, 'cut2': 0.0014652779262902783, 'cut3': 0.0014652779262902783, 'cut4': 0.0014652779262902783, 'cut5': 0.0014652779262902783, 'cut6': 0.0014652779262902783, 'cut7': 0.0014588619616132373, 'cut8': 0.0013158341536968093, 'cut9': 0.0005258875783882676, 'cut10': 2.8350982945221117e-06}), 'bkg_2022_WJets_WJetsToLNu5': defaultdict(<class 'float'>, {'all': 3.942011044451162e-07, 'hasVtx': 2.0454874566432025e-08, 'cut1': 2.0454874566432025e-08, 'cut2': 2.0454874566432025e-08, 'cut3': 2.0454874566432025e-08, 'cut4': 2.0454874566432025e-08, 'cut5': 2.0454874566432025e-08, 'cut6': 1.3633067522762393e-08, 'cut7': 1.3633067522762393e-08, 'cut8': 1.3633067522762393e-08, 'cut9': 1.3633067522762393e-08, 'cut10': 0.0}), 'bkg_2022_WJets_WJetsToLNu4': defaultdict(<class 'float'>, {'all': 0.024379769422406036, 'hasVtx': 0.001

In [5]:
#total_bkg_counts= weighted events/raw events. Represents normalization factor.
total_bkg_counts_arr=[]
total_bkg_counts_arr_sum=0

for k in bkg_histo['cutflow'].keys():   
    
    total_bkg_counts = bkg_histo['cutflow_cts'][k]['all'] / bkg_histo['cutflow'][k]['all']
    print(f'\nxsec * lumi = {total_bkg_counts} for ', k)
    total_bkg_counts_arr.append(total_bkg_counts)
    
total_bkg_counts_arr_sum=sum(total_bkg_counts_arr)

print(f'\nxsec * lumi = {total_bkg_counts_arr_sum} for the sub process WJets')




xsec * lumi = 286530.783 for  bkg_2022_WJets_WJetsToLNu6

xsec * lumi = 195650393.39999995 for  bkg_2022_WJets_WJetsToLNu5

xsec * lumi = 2759066.079 for  bkg_2022_WJets_WJetsToLNu4

xsec * lumi = 1414.2570749999998 for  bkg_2022_WJets_WJetsToLNu3

xsec * lumi = 20590.66317 for  bkg_2022_WJets_WJetsToLNu2

xsec * lumi = 74783154.6 for  bkg_2022_WJets_WJetsToLNu1

xsec * lumi = 273501149.7822449 for the sub process WJets


In [9]:
#What do I need?
#I want to find ** for Nocuts of all the samples here.
#cutDesc: cut names
#cutflow: efficiency = selected events / total events cutflow[cutName] += np.sum(events.genWgt)/sum_wgt
#cutflow_cts: reweighted by xsec cutflow_counts[cutName] = xsec*lumi*cutflow[cutName]
#cutflow_nevts: raw counts cutflow_nevts[cutName] += len(events) -- will not be used.
#cutflow_vtx_matched: (for signal MC only) fraction that the selected vertex (lowest chi2) passes the truth-matching (gen-matching)

#nevents=raw number of events 100-- how many events pass cut X
#weight 

#cutflow_cts is a key in bkg_histo, which holds the number of events passing different selection cuts.

cfd=bkg_histo["cutflow_cts"]
cts_dict = {k:0 for k in cuts}  #cts dict is initialized as a dictionary with: Each cut name as a key and the initial value set to 0.

for k in cfd.keys():#loops over sample names
    for c in cuts:#loops over cut names
        cts_dict[c] += cfd[k][c]   #cfd[k][c] retrieves the number of events that passed cut c for sample k
    
        print(f'\n:{k}  xsec * lumi * eff at {cutNames[c]} = {cts_dict[c]}')  #The accumulated count (cts_dict[c]), represents the weighted event count after that cut.
    
        #tot_bkg_cts = cts_dict[c] # subprocess summed
    print("cts_dict[c]:",cts_dict[c])
    print ("total_bkg_counts_arr_sum:", total_bkg_counts_arr_sum)
    tot_bkg_effs = {c:cts_dict[c]/total_bkg_counts_arr_sum for c in cuts}
    #total_bkg_counts_arr_sum is assumed to be the total initial background event count before any cuts.
    #tot_bkg_effs=total background counts after the cut(for a sample)/Initial background count (for all samples)
    print(tot_bkg_effs)


:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at No cuts = 15198.748808579854

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at Baseline Selection = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at Pass MET Filters = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at HEM Veto = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at MET Trigger (120 GeV) = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at MET > 200 GeV = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at No b-tagged jets = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at Leading jet pT > 80 GeV = 419.8472315325697

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at Leading jet |eta| < 2.4 = 418.00886014995683

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at dPhi(MET,leading jet) > 2.0 = 377.0269903568891

:bkg_2022_WJets_WJetsToLNu6  xsec * lumi * eff at dPhi(MET,all jets) > 0.75 = 150.68297960556419

:bkg_20

In [7]:
tree=[]
sum_tree=0

main_dict={}

for c in cuts: #loop over cuts first
    for k in cfd.keys():  #loop over samples then
    
        element=tot_bkg_effs[c]
        
        
        print("List for ", c, "sample ", k, ":",element)
        
    main_dict[c]=sum(element)
    print(main_dict)
        



List for  all sample  bkg_2022_WJets_WJetsToLNu6 : 0.0007456324367154015
List for  all sample  bkg_2022_WJets_WJetsToLNu5 : 0.0007456324367154015
List for  all sample  bkg_2022_WJets_WJetsToLNu4 : 0.0007456324367154015
List for  all sample  bkg_2022_WJets_WJetsToLNu3 : 0.0007456324367154015
List for  all sample  bkg_2022_WJets_WJetsToLNu2 : 0.0007456324367154015
List for  all sample  bkg_2022_WJets_WJetsToLNu1 : 0.0007456324367154015


TypeError: 'numpy.float64' object is not iterable

In [ ]:
df = pd.DataFrame(list(main_dict.items()), columns=['Cut Name', 'Value'])
print(df)